# **Processing the data**

Melanjutkan contoh dari bab sebelumnya, berikut adalah cara kita melatih *sequence classifier* pada satu batch di PyTorch:

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Tentu saja, hanya melatih model dengan dua kalimat tidak akan menghasilkan hasil yang sangat baik. Untuk mendapatkan hasil yang lebih baik, Anda perlu menyiapkan dataset yang lebih besar.

Pada bagian ini, kita akan menggunakan dataset MRPC (Microsoft Research Paraphrase Corpus) sebagai contoh, yang diperkenalkan dalam sebuah makalah oleh William B. Dolan dan Chris Brockett. Dataset ini terdiri dari 5.801 pasangan kalimat, dengan label yang menunjukkan apakah kalimat-kalimat tersebut merupakan parafrase atau tidak (yaitu, apakah kedua kalimat tersebut memiliki makna yang sama). Kami memilihnya untuk bab ini karena dataset ini kecil, sehingga mudah untuk bereksperimen dengan melatihnya.

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

Seperti yang dapat kita lihat, kita mendapatkan objek DatasetDict yang berisi set pelatihan, set validasi, dan set uji. Masing-masing berisi beberapa kolom (sentence1, sentence2, label, dan idx) dan jumlah baris yang bervariasi, yang merupakan jumlah elemen di setiap set (jadi, ada 3.668 pasangan kalimat di set pelatihan, 408 di set validasi, dan 1.725 di set uji).

Perintah ini mengunduh dan menyimpan dataset, secara default di ~/.cache/huggingface/datasets. Ingat dari Bab 2 bahwa kita dapat menyesuaikan folder cache kita dengan mengatur variabel lingkungan HF_HOME.

Kita dapat mengakses setiap pasangan kalimat di objek raw_datasets kita dengan menggunakan indeks, seperti pada dictionary.

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Kita dapat melihat bahwa label sudah berupa angka bulat, jadi kita tidak perlu melakukan preprocessing di sana. Untuk mengetahui angka bulat mana yang sesuai dengan label mana, kita dapat memeriksa fitur dari raw_train_dataset kita. Ini akan memberi tahu kita tipe dari setiap kolom:

In [ ]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

# **Preprocessing a dataset**

Untuk memproses dataset, kita perlu mengonversi teks menjadi angka yang dapat dipahami oleh model. Seperti yang kita lihat di bab sebelumnya, ini dilakukan dengan tokenizer. Kita dapat memberi tokenizer satu kalimat atau daftar kalimat, sehingga kita bisa langsung men-tokenize semua kalimat pertama dan semua kalimat kedua dari setiap pasangan seperti ini:

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

Namun, kita tidak bisa begitu saja memberikan dua urutan ke model dan mendapatkan prediksi apakah kedua kalimat tersebut merupakan parafrase atau bukan. Kita perlu menangani kedua urutan tersebut sebagai pasangan, dan menerapkan preprocessing yang sesuai. Untungnya, tokenizer juga dapat menerima pasangan urutan dan mempersiapkannya sesuai dengan yang diharapkan oleh model BERT kita:

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Kita telah membahas kunci *input_ids* dan *attention_mask* di Bab 2, tetapi kita menunda pembahasan tentang *token_type_ids*. Dalam contoh ini, inilah yang memberi tahu model bagian mana dari input yang merupakan kalimat pertama dan mana yang merupakan kalimat kedua.

Jika kita mendecode ID di dalam *input_ids* kembali menjadi kata-kata, kita akan mendapatkan seperti ini:

In [ ]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Jadi, kita melihat bahwa model mengharapkan input dalam bentuk [CLS] sentence1 [SEP] sentence2 [SEP] ketika ada dua kalimat. Menyelaraskan ini dengan *token_type_ids* memberi kita:



```
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]
```



Seperti yang kita lihat, bagian dari input yang sesuai dengan [CLS] sentence1 [SEP] semua memiliki *token type ID* 0, sementara bagian lainnya, yang sesuai dengan sentence2 [SEP], semua memiliki *token type ID* 1.

Perlu dicatat bahwa jika kita memilih checkpoint yang berbeda, kita tidak selalu akan mendapatkan *token_type_ids* dalam input yang sudah di-tokenize (misalnya, mereka tidak dikembalikan jika kita menggunakan model DistilBERT). *Token_type_ids* hanya dikembalikan ketika model tahu apa yang harus dilakukan dengan mereka, karena model tersebut telah melihatnya selama pretraining-nya.

Di sini, BERT dilatih dengan *token type IDs*, dan di atas tujuan *masked language modeling* yang kita bahas di Bab 1, ada tujuan tambahan yang disebut *next sentence prediction*. Tujuan dari tugas ini adalah untuk memodelkan hubungan antara pasangan kalimat.

Dengan *next sentence prediction*, model diberikan pasangan kalimat (dengan token yang secara acak disembunyikan) dan diminta untuk memprediksi apakah kalimat kedua mengikuti kalimat pertama. Untuk membuat tugas ini tidak sepele, setengah dari kalimat tersebut mengikuti satu sama lain dalam dokumen asli tempat kalimat tersebut diambil, dan setengah lainnya kalimat tersebut berasal dari dua dokumen yang berbeda.

Secara umum, kita tidak perlu khawatir apakah ada *token_type_ids* dalam input yang sudah di-tokenize: selama kita menggunakan checkpoint yang sama untuk tokenizer dan model, semuanya akan baik-baik saja karena tokenizer tahu apa yang harus diberikan kepada modelnya.

Sekarang setelah kita melihat bagaimana tokenizer kita dapat menangani satu pasangan kalimat, kita dapat menggunakannya untuk men-tokenize seluruh dataset kita: seperti di bab sebelumnya, kita dapat memberi tokenizer daftar pasangan kalimat dengan memberinya daftar kalimat pertama, kemudian daftar kalimat kedua. Ini juga kompatibel dengan opsi *padding* dan *truncation* yang kita lihat di Bab 2. Jadi, salah satu cara untuk memproses dataset pelatihan adalah:

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

Ini berjalan dengan baik, tetapi memiliki kekurangan karena mengembalikan sebuah dictionary (dengan kunci-kunci *input_ids*, *attention_mask*, dan *token_type_ids*, serta nilai-nilai yang berupa *lists of lists*). Ini juga hanya akan berfungsi jika Anda memiliki cukup RAM untuk menyimpan seluruh dataset selama tokenisasi (sementara dataset dari pustaka Hugging Face Datasets disimpan dalam file Apache Arrow di disk, sehingga Anda hanya memuat sampel yang diminta ke dalam memori).

Untuk menjaga data tetap sebagai sebuah dataset, kita akan menggunakan metode *Dataset.map()*. Ini juga memberikan kita fleksibilitas tambahan, jika kita memerlukan pemrosesan lebih lanjut selain hanya tokenisasi. Metode *map()* bekerja dengan menerapkan sebuah fungsi pada setiap elemen dari dataset, jadi mari kita definisikan sebuah fungsi yang akan men-tokenisasi input kita:

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Fungsi ini mengambil sebuah dictionary (seperti item dari dataset kita) dan mengembalikan sebuah dictionary baru dengan kunci *input_ids*, *attention_mask*, dan *token_type_ids*. Perhatikan bahwa ini juga berfungsi jika dictionary contoh berisi beberapa sampel (setiap kunci sebagai daftar kalimat) karena tokenizer bekerja pada daftar pasangan kalimat, seperti yang kita lihat sebelumnya. Ini akan memungkinkan kita untuk menggunakan opsi *batched=True* dalam panggilan kita ke *map()*, yang akan mempercepat tokenisasi secara signifikan. Tokenizer ini didukung oleh tokenizer yang ditulis dalam Rust dari pustaka Hugging Face Tokenizers. Tokenizer ini bisa sangat cepat, tetapi hanya jika kita memberinya banyak input sekaligus.

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Cara pustaka Hugging Face Datasets menerapkan pemrosesan ini adalah dengan menambahkan field baru ke dalam dataset, satu untuk setiap kunci dalam dictionary yang dikembalikan oleh fungsi pemrosesan:

# **Dynamic padding**

Fungsi yang bertanggung jawab untuk menyusun sampel dalam satu batch disebut fungsi collate. Fungsi ini dapat diteruskan saat membangun DataLoader, dengan fungsi default yang mengubah sampel menjadi tensor PyTorch dan menggabungkannya (rekursif jika elemen berupa lists, tuples, atau dictionaries). Karena input kita memiliki ukuran yang berbeda, kita menunda padding hingga batch tertentu. Ini mempercepat pelatihan, tetapi perlu diperhatikan bahwa TPUs lebih suka bentuk tetap, meskipun memerlukan padding tambahan.

Untuk itu, kita perlu mendefinisikan fungsi collate yang akan menambahkan padding yang tepat. Untungnya, pustaka Hugging Face Transformers menyediakan fungsi ini melalui DataCollatorWithPadding, yang membutuhkan tokenizer saat diinstansiasi untuk mengetahui token padding yang digunakan dan posisi padding, serta melakukan semuanya secara otomatis.

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Untuk menguji alat baru ini, mari kita ambil beberapa sampel dari set pelatihan yang ingin kita gabungkan dalam satu batch. Di sini, kita menghapus kolom idx, sentence1, dan sentence2 karena tidak diperlukan dan berisi string (dan kita tidak bisa membuat tensor dengan string), lalu kita lihat panjang setiap entri dalam batch:

In [ ]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

Kita mendapatkan sampel dengan panjang yang bervariasi, mulai dari 32 hingga 67. Padding dinamis berarti sampel dalam batch ini harus dipad dengan panjang 67, panjang maksimum dalam batch tersebut. Tanpa padding dinamis, semua sampel harus dipad dengan panjang maksimum dalam seluruh dataset, atau panjang maksimum yang bisa diterima oleh model. Mari kita periksa kembali apakah data_collator kita telah melakukan padding dinamis pada batch dengan benar:

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

Sekarang setelah kita mengubah teks mentah menjadi batch yang dapat diproses oleh model, kita siap untuk melakukan fine-tuning!

# **Fine-tuning a model with the Trainer API**

Hugging Face Transformers menyediakan trainer class untuk membantu Anda melakukan fine-tuning pada model pretrained apa pun yang disediakan di dataset Anda. Setelah melakukan semua pekerjaan preprocessing data di bagian sebelumnya, Anda hanya perlu beberapa langkah lagi untuk mendefinisikan Trainer. Bagian yang paling sulit mungkin adalah mempersiapkan lingkungan untuk menjalankan Trainer.train(), karena proses ini akan sangat lambat di CPU. Jika Anda tidak memiliki GPU, Anda dapat mengakses GPU atau TPU gratis di Google Colab.

Contoh kode di bawah ini mengasumsikan Anda telah menjalankan contoh di bagian sebelumnya. Berikut adalah ringkasan singkat tentang apa yang Anda butuhkan:

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

# **Training**

Langkah pertama sebelum kita dapat mendefinisikan Trainer adalah mendefinisikan trainingArguments class yang akan berisi semua hyperparameter yang akan digunakan Trainer untuk pelatihan dan evaluasi. Satu-satunya argumen yang perlu kita berikan adalah direktori tempat model yang telah dilatih akan disimpan, serta checkpoint selama proses pelatihan. Untuk yang lainnya, kita dapat membiarkan nilai default, yang seharusnya bekerja dengan baik untuk fine-tuning dasar.

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments("test-trainer")

Langkah kedua adalah mendefinisikan model kita. Seperti di bab sebelumnya, kita akan menggunakan AutoModelForSequenceClassification class, dengan dua label:

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kita akan memperhatikan bahwa, berbeda dengan di Bab 2, kita mendapatkan peringatan setelah menginstansiasi model pra-latih ini. Hal ini karena BERT tidak dilatih sebelumnya untuk mengklasifikasikan pasangan kalimat, sehingga kepala model pra-latih telah dibuang dan kepala baru yang sesuai untuk klasifikasi urutan telah ditambahkan sebagai gantinya. Peringatan tersebut menunjukkan bahwa beberapa bobot tidak digunakan (yang sesuai dengan kepala pra-latih yang dibuang) dan beberapa lainnya diinisialisasi secara acak (untuk kepala baru). Peringatan ini mengakhiri dengan mendorong kita untuk melatih model, yang memang akan kita lakukan sekarang.

Setelah kita memiliki model, kita bisa mendefinisikan Trainer dengan memberikannya semua objek yang telah dibangun hingga saat ini — model, training_args, dataset pelatihan dan validasi, data_collator, dan tokenizer kita:

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-29-6179b5eb883d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Perhatikan bahwa ketika kita memberikan tokenizer seperti yang kita lakukan di sini, data_collator default yang digunakan oleh Trainer akan menjadi DataCollatorWithPadding seperti yang telah didefinisikan sebelumnya, sehingga kita bisa melewatkan baris `data_collator=data_collator` dalam pemanggilan ini. Meskipun begitu, masih penting untuk menunjukkan bagian pemrosesan ini di bagian 2!

Untuk melakukan fine-tune model pada dataset kita, kita hanya perlu memanggil metode `train()` dari Trainer kita:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.544900
1000,0.369700


TrainOutput(global_step=1377, training_loss=0.3972386101838377, metrics={'train_runtime': 272.4149, 'train_samples_per_second': 40.394, 'train_steps_per_second': 5.055, 'total_flos': 405114969714960.0, 'train_loss': 0.3972386101838377, 'epoch': 3.0})

Ini akan memulai fine-tuning (yang seharusnya memakan waktu beberapa menit di GPU) dan melaporkan training loss setiap 500 langkah. Namun, ini tidak akan memberitahukan seberapa baik (atau buruk) performa model kita. Hal ini karena:

1. Kita tidak memberitahu Trainer untuk melakukan evaluasi selama pelatihan dengan mengatur `evaluation_strategy` ke "steps" (evaluasi setiap `eval_steps`) atau "epoch" (evaluasi di akhir setiap epoch).
2. Kita tidak menyediakan Trainer dengan fungsi `compute_metrics()` untuk menghitung metrik selama evaluasi tersebut (sehingga evaluasi hanya akan mencetak loss, yang bukan angka yang sangat intuitif).

# **Evaluation**

Mari kita lihat bagaimana kita dapat membangun fungsi `compute_metrics()` yang berguna dan menggunakannya saat kita melakukan pelatihan berikutnya. Fungsi ini harus menerima objek `EvalPrediction` (yang merupakan tuple bernama dengan field `predictions` dan `label_ids`) dan akan mengembalikan sebuah dictionary yang memetakan string ke float (string tersebut adalah nama metrik yang dikembalikan, dan float adalah nilai metrik tersebut). Untuk mendapatkan beberapa prediksi dari model kita, kita bisa menggunakan perintah `Trainer.predict()`:

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


Output dari metode `predict()` adalah tuple bernama lain dengan tiga field: `predictions`, `label_ids`, dan `metrics`. Field `metrics` hanya akan berisi loss pada dataset yang diteruskan, serta beberapa metrik waktu (berapa lama waktu yang dibutuhkan untuk memprediksi, secara total dan rata-rata). Setelah kita menyelesaikan fungsi `compute_metrics()` dan memberikannya ke `Trainer`, field tersebut juga akan berisi metrik yang dikembalikan oleh `compute_metrics()`.

Seperti yang dapat Anda lihat, `predictions` adalah array dua dimensi dengan bentuk 408 x 2 (408 adalah jumlah elemen dalam dataset yang kita gunakan). Itu adalah logits untuk setiap elemen dataset yang kita kirim ke `predict()` (seperti yang Anda lihat di bab sebelumnya, semua model Transformer mengembalikan logits). Untuk mengubahnya menjadi prediksi yang bisa kita bandingkan dengan label kita, kita perlu mengambil indeks dengan nilai maksimum di sumbu kedua:

In [ ]:
import numpy as np

In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

Sekarang kita bisa membandingkan `preds` dengan label. Untuk membangun fungsi `compute_metric()`, kita akan mengandalkan metrik dari Hugging Face Evaluate library. Kita bisa memuat metrik yang terkait dengan dataset MRPC dengan mudah seperti kita memuat dataset, kali ini menggunakan fungsi `evaluate.load()`. Objek yang dikembalikan memiliki metode `compute()` yang bisa kita gunakan untuk melakukan perhitungan metrik:

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8627450980392157, 'f1': 0.9027777777777778}

Hasil yang tepat yang Anda dapatkan mungkin bervariasi, karena inisialisasi acak dari kepala model dapat mengubah metrik yang dicapai. Di sini, kita dapat melihat bahwa model kita memiliki akurasi 86% pada set validasi dan skor F1 90. Itu adalah dua metrik yang digunakan untuk mengevaluasi hasil pada dataset MRPC untuk benchmark GLUE. Tabel dalam makalah BERT melaporkan skor F1 88,9 untuk model dasar. Itu adalah model tanpa huruf besar, sementara kita saat ini menggunakan model dengan huruf besar, yang menjelaskan hasil yang lebih baik.

Menggabungkan semuanya, kita mendapatkan fungsi `compute_metrics()`:

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Dan untuk melihatnya digunakan dalam tindakan untuk melaporkan metrik di akhir setiap epoch, berikut adalah cara kita mendefinisikan Trainer baru dengan fungsi `compute_metrics()` ini:

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-39-415be425bb02>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Perhatikan bahwa kita membuat `TrainingArguments` baru dengan `evaluation_strategy` disetel ke "epoch" dan model baru — jika tidak, kita hanya akan melanjutkan pelatihan model yang sudah kita latih. Untuk memulai sesi pelatihan baru, kita jalankan:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.371126,0.857843,0.901024
2,0.527200,0.459970,0.845588,0.893401
3,0.279600,0.774146,0.848039,0.894915


TrainOutput(global_step=1377, training_loss=0.3341553964386319, metrics={'train_runtime': 262.6475, 'train_samples_per_second': 41.896, 'train_steps_per_second': 5.243, 'total_flos': 405114969714960.0, 'train_loss': 0.3341553964386319, 'epoch': 3.0})

Kali ini, ia akan melaporkan loss validasi dan metrik di akhir setiap epoch di samping loss pelatihan. Sekali lagi, akurasi/F1 score yang Anda capai mungkin sedikit berbeda dari yang saya temukan, karena inisialisasi acak pada kepala model, tetapi harusnya berada di kisaran yang sama.

# **A full training**

Sekarang kita akan melihat bagaimana mencapai hasil yang sama seperti di bagian terakhir tanpa menggunakan kelas Trainer. Sekali lagi, saya mengasumsikan anda telah melakukan pemrosesan data di bagian 2. Berikut adalah ringkasan singkat yang mencakup semua yang anda perlukan:

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

# **Prepare for training**

Sebelum menulis loop pelatihan kita, kita perlu mendefinisikan beberapa objek. Yang pertama adalah dataloader yang akan kita gunakan untuk iterasi atas batch. Tetapi sebelum kita mendefinisikan dataloader tersebut, kita perlu menerapkan sedikit post-processing pada tokenized_datasets, untuk menangani beberapa hal yang sebelumnya dilakukan otomatis oleh Trainer. Secara khusus, kita perlu:

- Menghapus kolom yang berisi nilai yang tidak diharapkan oleh model (seperti kolom sentence1 dan sentence2).
- Mengganti nama kolom label menjadi labels (karena model mengharapkan argumen tersebut bernama labels).
- Menetapkan format dataset sehingga mereka mengembalikan tensor PyTorch alih-alih lists.

Tokenized_datasets kita memiliki satu metode untuk setiap langkah tersebut:

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Sekarang setelah ini selesai, kita dapat dengan mudah mendefinisikan dataloaders kita:

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

Untuk dengan cepat memeriksa apakah ada kesalahan dalam pemrosesan data, kita bisa memeriksa satu batch seperti ini:

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

Sekarang kita telah sepenuhnya menyelesaikan pemrosesan data (sebuah pencapaian yang memuaskan namun sulit dicapai bagi praktisi ML), mari kita beralih ke model. Kita menginstansiasi modelnya persis seperti yang kita lakukan di bagian sebelumnya:

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Untuk memastikan bahwa semuanya berjalan lancar selama pelatihan, kita mengirimkan batch ini ke model:

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.9363, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


Semua model Transformers dari Hugging Face akan mengembalikan loss ketika label diberikan, dan kita juga mendapatkan logits (dua untuk setiap input dalam batch kita, sehingga tensor berukuran 8 x 2).

Kita hampir siap untuk menulis loop pelatihan kita! Kita hanya kekurangan dua hal: optimizer dan scheduler laju pembelajaran. Karena kita mencoba mereplikasi apa yang dilakukan Trainer secara manual, kita akan menggunakan default yang sama. Optimizer yang digunakan oleh Trainer adalah AdamW, yang merupakan versi dari Adam, tetapi dengan penyesuaian untuk regularisasi decay bobot (lihat "Decoupled Weight Decay Regularization" oleh Ilya Loshchilov dan Frank Hutter).

In [ ]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Akhirnya, scheduler laju pembelajaran yang digunakan secara default adalah decay linier dari nilai maksimum (5e-5) ke 0. Untuk mendefinisikannya dengan benar, kita perlu mengetahui jumlah langkah pelatihan yang akan dilakukan, yang merupakan jumlah epoch yang ingin kita jalankan dikalikan dengan jumlah batch pelatihan (yaitu panjang dataloader pelatihan kita). Trainer menggunakan tiga epoch secara default, jadi kita akan mengikuti itu.

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


# **The training loop**

Satu hal terakhir: kita ingin menggunakan GPU jika kita memiliki akses ke satu (pada CPU, pelatihan bisa memakan waktu beberapa jam alih-alih beberapa menit). Untuk melakukan ini, kita mendefinisikan perangkat tempat kita akan meletakkan model dan batch kita:

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

Sekarang kita siap untuk melatih! Untuk mendapatkan gambaran kapan pelatihan akan selesai, kita menambahkan progress bar di atas jumlah langkah pelatihan kita, menggunakan pustaka tqdm:

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

Anda dapat melihat bahwa inti dari loop pelatihan sangat mirip dengan yang ada di pengantar. Kami tidak meminta pelaporan apa pun, jadi loop pelatihan ini tidak akan memberi tahu kami apa pun tentang kinerja model. Kami perlu menambahkan loop evaluasi untuk itu.

# **The evaluation loop**

Seperti yang kita lakukan sebelumnya, kita akan menggunakan metrik yang disediakan oleh pustaka Evaluate dari Hugging Face. Kita sudah melihat metode `metric.compute()`, tetapi metrik sebenarnya dapat mengakumulasi batch untuk kita saat kita melewati loop prediksi dengan metode `add_batch()`. Setelah kita mengakumulasi semua batch, kita dapat mendapatkan hasil akhir dengan `metric.compute()`. Berikut adalah cara mengimplementasikan semua ini dalam loop evaluasi:

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8725490196078431, 'f1': 0.910958904109589}

Sekali lagi, hasil Anda mungkin sedikit berbeda karena adanya ketidakpastian dalam inisialisasi kepala model dan pengacakan data, tetapi hasilnya seharusnya berada dalam kisaran yang sama.

# **Supercharge your training loop with Hugging Face Accelerate**

Loop pelatihan yang kita definisikan sebelumnya berfungsi dengan baik pada satu CPU atau GPU. Namun, dengan menggunakan pustaka Hugging Face Accelerate, hanya dengan beberapa penyesuaian kita dapat mengaktifkan pelatihan terdistribusi pada beberapa GPU atau TPU. Dimulai dari pembuatan dataloader untuk pelatihan dan validasi, berikut adalah bagaimana loop pelatihan manual kita terlihat:

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

Ini adalah bentuk code jika menggunakan Accelerate:

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

Menempatkan ini dalam skrip `train.py` akan membuat skrip tersebut dapat dijalankan pada pengaturan terdistribusi apa pun. Untuk mencobanya dalam pengaturan terdistribusi Anda, jalankan perintah:

In [ ]:
!accelerate config

----------------------------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)

This machine
----------------------------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-MUSA
    TPU
TPU
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:yes
Do you want to use Intel PyTorch Extension (IPEX) to speed up training on CPU? [yes/NO]:yes
Do you wish to optimize your script with torch dynamo?[yes/NO]:yes
----------------------------------------------------------------------------------------------------W

yang akan meminta Anda untuk menjawab beberapa pertanyaan dan menyimpan jawaban Anda dalam file konfigurasi yang digunakan oleh perintah ini:

In [ ]:
!accelerate launch --debug train.py


Traceback (most recent call last):
  File "/content/train.py", line 6, in <module>
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
NameError: name 'checkpoint' is not defined. Did you mean: 'breakpoint'?
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 1168, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 763, in simple_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/usr/bin/python3', 'train.py']' returned non-zero exit status 1.


yang akan meluncurkan pelatihan terdistribusi.

Jika Anda ingin mencoba ini di Notebook (misalnya, untuk mengujinya dengan TPU di Colab), cukup tempelkan kode dalam sebuah fungsi `training_function()` dan jalankan sel terakhir dengan:

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)